In [3]:
import pandas as pd
from tqdm import tqdm
!unzip DATA/1014_4361_bundle_archive.zip -d DATA/

Archive:  DATA/1014_4361_bundle_archive.zip
  inflating: DATA/ner.csv            
  inflating: DATA/ner_dataset.csv    


In [2]:
df = pd.read_csv("DATA/ner.csv", encoding = "ISO-8859-1", error_bad_lines=False)

b'Skipping line 281837: expected 25 fields, saw 34\n'


In [3]:
df.head()

,Unnamed: 0,lemma,next-lemma,next-next-lemma,next-next-pos,next-next-shape,next-next-word,next-pos,next-shape,next-word,...,prev-prev-lemma,prev-prev-pos,prev-prev-shape,prev-prev-word,prev-shape,prev-word,sentence_idx,shape,word,tag
0,0,thousand,of,demonstr,NNS,lowercase,demonstrators,IN,lowercase,of,...,__start2__,__START2__,wildcard,__START2__,wildcard,__START1__,1.0,capitalized,Thousands,O
1,1,of,demonstr,have,VBP,lowercase,have,NNS,lowercase,demonstrators,...,__start1__,__START1__,wildcard,__START1__,capitalized,Thousands,1.0,lowercase,of,O
2,2,demonstr,have,march,VBN,lowercase,marched,VBP,lowercase,have,...,thousand,NNS,capitalized,Thousands,lowercase,of,1.0,lowercase,demonstrators,O
3,3,have,march,through,IN,lowercase,through,VBN,lowercase,marched,...,of,IN,lowercase,of,lowercase,demonstrators,1.0,lowercase,have,O
4,4,march,through,london,NNP,capitalized,London,IN,lowercase,through,...,demonstr,NNS,lowercase,demonstrators,lowercase,have,1.0,lowercase,marched,O


In [4]:
df.columns

Index(['Unnamed: 0', 'lemma', 'next-lemma', 'next-next-lemma', 'next-next-pos',
       'next-next-shape', 'next-next-word', 'next-pos', 'next-shape',
       'next-word', 'pos', 'prev-iob', 'prev-lemma', 'prev-pos',
       'prev-prev-iob', 'prev-prev-lemma', 'prev-prev-pos', 'prev-prev-shape',
       'prev-prev-word', 'prev-shape', 'prev-word', 'sentence_idx', 'shape',
       'word', 'tag'],
      dtype='object')

In [5]:
df=df.drop(['Unnamed: 0', 'lemma', 'next-lemma', 'next-next-lemma', 'next-next-pos',
       'next-next-shape', 'next-next-word', 'next-pos', 'next-shape',
       'next-word', 'prev-iob', 'prev-lemma', 'prev-pos',
       'prev-prev-iob', 'prev-prev-lemma', 'prev-prev-pos', 'prev-prev-shape',
       'prev-prev-word', 'prev-shape', 'prev-word',"pos"],axis=1)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050795 entries, 0 to 1050794
Data columns (total 4 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   sentence_idx  1050794 non-null  float64
 1   shape         1050794 non-null  object 
 2   word          1050794 non-null  object 
 3   tag           1050794 non-null  object 
dtypes: float64(1), object(3)
memory usage: 32.1+ MB


In [7]:
df.describe()

,sentence_idx
count,1.050794e+06
mean,1.898184e+04
std,1.576237e+04
min,1.000000e+00
25%,5.997000e+03
50%,1.201700e+04
75%,3.592600e+04
max,4.795900e+04


In [8]:
df.head()

,sentence_idx,shape,word,tag
0,1.0,capitalized,Thousands,O
1,1.0,lowercase,of,O
2,1.0,lowercase,demonstrators,O
3,1.0,lowercase,have,O
4,1.0,lowercase,marched,O


In [9]:
df['tag'].describe()

count     1050794
unique         17
top             O
freq       889973
Name: tag, dtype: object

In [10]:
df.tag.unique()

array(['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim',
       'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat', nan], dtype=object)

In [11]:
df.head(25)

,sentence_idx,shape,word,tag
0,1.0,capitalized,Thousands,O
1,1.0,lowercase,of,O
2,1.0,lowercase,demonstrators,O
3,1.0,lowercase,have,O
4,1.0,lowercase,marched,O
5,1.0,lowercase,through,O
6,1.0,capitalized,London,B-geo
7,1.0,lowercase,to,O
8,1.0,lowercase,protest,O
9,1.0,lowercase,the,O


In [12]:
total_sentences = dict()
for i in df.itertuples():
    if i[1] not in total_sentences:
        total_sentences[i[1]] = i[0]

In [13]:
len(total_sentences), min(total_sentences.keys()), max(total_sentences.keys())

(35178, 1.0, 47959.0)

In [14]:
df.iloc[3].tag, df.iloc[3].word, df.iloc[3]['shape'], df.iloc[3].sentence_idx

('O', 'have', 'lowercase', 1.0)

In [16]:
final_sentences = []
for i in tqdm(total_sentences):
    sentence = []
    present_id = i
    idx = total_sentences[i]
    try:
        while df.iloc[idx].sentence_idx == present_id:
            sentence.append((df.iloc[idx].word, df.iloc[idx]['shape'],  
                             df.iloc[idx].tag))
            idx += 1
    except: break
    final_sentences.append(sentence)    

100%|█████████▉| 35177/35178 [07:02<00:00, 83.28it/s] 


In [17]:
len(final_sentences)

35177

In [18]:
final_sentences[0]

[('Thousands', 'capitalized', 'O'),
 ('of', 'lowercase', 'O'),
 ('demonstrators', 'lowercase', 'O'),
 ('have', 'lowercase', 'O'),
 ('marched', 'lowercase', 'O'),
 ('through', 'lowercase', 'O'),
 ('London', 'capitalized', 'B-geo'),
 ('to', 'lowercase', 'O'),
 ('protest', 'lowercase', 'O'),
 ('the', 'lowercase', 'O'),
 ('war', 'lowercase', 'O'),
 ('in', 'lowercase', 'O'),
 ('Iraq', 'capitalized', 'B-geo'),
 ('and', 'lowercase', 'O'),
 ('demand', 'lowercase', 'O'),
 ('the', 'lowercase', 'O'),
 ('withdrawal', 'lowercase', 'O'),
 ('of', 'lowercase', 'O'),
 ('British', 'capitalized', 'B-gpe'),
 ('troops', 'lowercase', 'O'),
 ('from', 'lowercase', 'O'),
 ('that', 'lowercase', 'O'),
 ('country', 'lowercase', 'O'),
 ('.', 'punct', 'O')]

In [19]:
import pickle as pkl

In [20]:
pkl.dump(final_sentences, open('DATA/training_instances.pkl', 'wb'))